# Import Dependencies

In [1]:
import os 
import numpy as np 
import cv2 as cv
import matplotlib.pyplot as plt

from pygments.formatters import img
from tqdm import tqdm

from preprocessing.edge_extraction import *
from feature_extraction import * 
from preprocessing.fourier_transform import * 
from preprocessing.image_conversion import * 
from clustering import *
from preprocessing.contrast_enhancement import *

# Pre-processing

To reduce noise in images of whole artworks and fragments, we initially considered using the Fourier transform to process the images in the frequency domain.

While converting an image from RGBA to grayscale simplifies processing, it results in the loss of RGB color and alpha channel data, which can be problematic if that information is needed later. Therefore, we chose to split the image into its primary color channels (excluding the alpha channel) and process each channel separately in the frequency domain. After filtering, we planned to reconstruct the filtered image by recombining the processed channels.

However, after several trials, we found that processing the channels separately led to significant information loss in one or more channels. Consequently, we decided to use the NLMeansDenoising filter instead.

Since our goal is to cluster fragments that belong to the same image, we focus on maintaining "continuity" along the fragment borders. Therefore, our process emphasizes the information present along these edges.

Steps:
1. Extract a working region from the borders of the fragment.
2. Filter out the transparent pixels from the working region.
3. Denoise the working region.

**CONSIDERATION**: Contrast enhancement.

In [ ]:
images = create_dataset("./data", threshold=5)

# Feature Extraction

To extract relevant features from the fragments, we employ two methods:
- Color Histograms
- Gradient Jacobians

## Color Histograms

Color histograms are graphical representations of the distribution of colors in an image. They quantify the number of pixels that have specific color values, effectively capturing the color composition of the image. By analyzing the color histograms of image fragments, we can compare and cluster similar fragments based on their color distributions.

**This technique is particularly useful for identifying and matching regions of images that share similar color patterns**.

In [ ]:
flatten_color_histograms = compute_color_histograms(images)
unflatten_color_histograms = compute_color_histograms(images, flatten=False)

In [ ]:
distance_matrix_color_histogram = compute_color_histogram_dist_matrix(unflatten_color_histograms)
distance_matrix_color_histogram

### K-Means

In [76]:
def f12(precision_score: float, recall_score: float) -> float:
    """
    Calculates F1 score given precision and recall.

    Args:
        precision_score (float): Precision score.
        recall_score (float): Recall score.

    Returns:
        float: F1 score.
    """
    return 2 * (precision_score * recall_score) / (precision_score + recall_score) if (precision_score + recall_score) else 0

def recall2(reference_image_id: int, root_dir: str, cluster_dirs_exp: List[str], ext: str = ".png") -> float:
    """
    Calculates recall given a reference image ID, root directory, and an excluded cluster directory.

    Args:
        reference_image_id (int): The ID of the reference image.
        root_dir (str): Root directory containing subdirectories.
        cluster_dirs_exp (List[str]): Names of the excluded cluster directories.
        ext (str, optional): File extension to filter images (default is ".png").

    Returns:
        float: Recall score (true positives / (true positives + false negatives)).
    """
    tp = 0
    for cluster_dir in cluster_dirs_exp:
        for root, _, files in os.walk(os.path.join(root_dir, cluster_dir)):
            for filename in files:
                if not filename.endswith(ext):
                    continue
                if filename.split(".")[1] == str(reference_image_id):
                    tp += 1

    fn = 0
    for dirpath, dirnames, filenames in os.walk(root_dir):
        dirnames[:] = [d for d in dirnames if d not in cluster_dirs_exp]
        for filename in filenames:
            if not filename.endswith(ext):
                continue
            if filename.split(".")[1] == str(reference_image_id):
                fn += 1
    total = tp + fn
    
    return tp / total if total else 0

def compute_metrics2(reference_image_id: int, root_dir: str, ext: str = ".png", output_file: str = None) -> dict:
    """
    Calculates precision scores for each cluster directory given a reference image ID and a root directory containing images.
    Also calculates the recall for the cluster directory with the highest precision.

    Args:
        reference_image_id (int): The ID of the reference image.
        root_dir (str): Path to the directory containing the clusters.
        ext (str, optional): File extension to filter images (default is ".png").
        output_file (str, optional): Path to the file where metrics will be saved (default is None).

    Returns:
        dict: A dictionary containing:
            - "max_item": A tuple with the directory having the highest precision score and the score itself.
            - "precision_scores": A dictionary with precision scores for each cluster directory.
            - "recall": The recall score for the cluster directory with the highest precision.
    """
    f1_scores = {}
    first_dir = True

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if first_dir:
            first_dir = False
            continue
        tp = 0
        for filename in filenames:
            if not filename.endswith(ext):
                continue
            if filename.split(".")[1] == str(reference_image_id):
                tp += 1
        precision = tp / len(filenames) if filenames else 0
        recall = recall2(reference_image_id=reference_image_id, root_dir=root_dir,
                         cluster_dirs_exp=[dirpath.split(os.path.sep)[-1]], ext=ext)
        f1_scores[dirpath.split(os.path.sep)[-1]] = f12(precision, recall)

    max_value = max(f1_scores.values())
    cluster_dirs = [dirpath for dirpath, score in f1_scores.items() if score == max_value]
    max_items = [(dirpath, score) for dirpath, score in f1_scores.items() if score == max_value]

    metrics = {
        "max_items": max_items,
        "f1_scores": f1_scores,
    }

    if output_file is not None:
        json.dump(metrics, open(output_file, "w"))
    return metrics

In [61]:
references_ids = [reference.split(".")[1] for reference in os.listdir("./references")]
references_ids

['33', '34', '35', '36', '37', '38', '39', '40']

In [77]:
import pickle 
from sklearn.cluster import KMeans


os.makedirs("./optimal_data", exist_ok=True)

# while len(references) > 0:
# images = create_dataset("./data", threshold=5)
# flatten_color_histograms = compute_color_histograms(images)
# kmeans = KMeans(n_clusters=len(references), random_state=42)
# fit_kmeans = kmeans.fit(flatten_color_histograms)

# create_cluster_dirs(data_dir="./data", output_dir="clusters/kmeans/colors", labels=fit_kmeans.labels_)
scores = {}
for reference_id in references_ids:
    scores[reference_id] = compute_metrics2(reference_id, "clusters/kmeans/colors")

threshold = 0.80
opt_clusters = {}
for reference_id, d in scores.items():
    max_items = d["max_items"]
    for max_item in max_items:
        if max_item[1] >= threshold:
            if reference_id in opt_clusters:
                opt_clusters[reference_id].append(max_item)
            else:
                opt_clusters[reference_id] = [max_item]
opt_clusters
            
# # move the optimal clusters to another path and reinitiate the clustering process without those fragments
# opt_dir = "optimal_clusters/kmeans/colors"
# os.makedirs(opt_dir, exist_ok=True)
# for reference_id, cluster_dirs in opt_clusters.items():
#     reference_dir = os.path.join(opt_dir, reference_id)
#     os.makedirs(reference_dir, exist_ok=True)
#     for cluster_dir in cluster_dirs:
#         img_dir = os.path.join("clusters/kmeans/colors", cluster_dir)
#         for filename in os.listdir(img_dir):
#             shutil.copy(os.path.join(img_dir, filename), os.path.join(reference_dir, filename))
#             shutil.move(os.path.join("./data", filename), os.path.join("./optimal_data", filename))
#         shutil.rmtree(img_dir)
#     references.remove(reference_id)

{'33': [('cluster_4', 0.8809523809523809)],
 '34': [('cluster_6', 0.8235294117647058)],
 '35': [('cluster_5', 0.9333333333333333)]}

In [4]:
restore_data("optimal_data", "data")

## Gradient Jacobians

Gradient Jacobians represent the gradients of pixel intensities in an image. They capture the rate of change of pixel values in both the horizontal and vertical directions, highlighting edges and texture details. By computing the Jacobians of image fragments, we can compare and group fragments that exhibit similar edge and texture patterns. Formally, the gradient jacobians we use are of the form:

$$
\begin{align}
\begin{bmatrix} G_x & G_{x\_gray} \\ G_y & G_{y\_gray} \end{bmatrix}
\end{align}
$$

where $G_x$ and $G_y$ are the aggregated gradient of the RGB channels, while $G_{x\_gray}$ and $G_{y\_gray}$ are the gradient of the grayscale image.

This method is especially valuable for identifying structural similarities and continuities between different fragments.

In [ ]:
flatten_jacobians = compute_jacobians(images)
unflatten_jacobians = compute_jacobians(images, flatten=False)

In [ ]:
distance_matrix_jacobians = compute_jacobians_dist_matrix(unflatten_jacobians)
distance_matrix_jacobians

### K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
fit_kmeans = kmeans.fit(flatten_jacobians)
create_cluster_dirs(data_dir="./data", output_dir="clusters/kmeans/jacobians", labels=fit_kmeans.labels_)

In [8]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

before = cv2.imread('data/5.38.35.png')
after = cv2.imread('references/5.37.jpg')

max_w = max(before.shape[0], after.shape[0])
max_h = max(before.shape[1], after.shape[1])

before = cv2.resize(before, (max_w, max_h))
after = cv2.resize(after, (max_w, max_h))

# Convert images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

cv2.imshow('before', before)
cv2.imshow('after', after)
cv2.imshow('diff',diff)
cv2.imshow('mask',mask)
cv2.imshow('filled after',filled_after)
cv2.waitKey(0)

Image similarity 0.2830785263439892


-1

In [3]:
from skimage.metrics import structural_similarity

fragments = create_dataset("data")
og_fragments = create_dataset("data", extract_borders=False)
references = []

for reference in os.listdir("references"):
    references.append(cv.imread(os.path.join("references", reference), cv.IMREAD_UNCHANGED))

references

Creating dataset: 100%|██████████| 328/328 [00:52<00:00,  6.20it/s]


[array([[[ 35,  54,  74, 255],
         [ 38,  56,  80, 255],
         [ 43,  67,  78, 255],
         ...,
         [ 43,  63,  77, 255],
         [ 36,  57,  69, 255],
         [ 26,  50,  58, 248]],
 
        [[ 31,  52,  72, 250],
         [ 38,  61,  75, 255],
         [ 39,  61,  79, 255],
         ...,
         [ 36,  60,  67, 250],
         [ 30,  56,  55, 255],
         [ 25,  51,  57, 244]],
 
        [[ 39,  61,  77, 253],
         [ 43,  68,  81, 255],
         [ 50,  73,  90, 255],
         ...,
         [ 30,  50,  56, 250],
         [ 23,  44,  47, 251],
         [ 19,  36,  44, 248]],
 
        ...,
 
        [[ 20,  45,  72, 255],
         [ 26,  51,  82, 255],
         [ 30,  56,  87, 253],
         ...,
         [ 39,  46,  71, 255],
         [ 24,  40,  65, 250],
         [ 23,  31,  56, 250]],
 
        [[ 25,  45,  75, 251],
         [ 32,  49,  74, 240],
         [ 30,  53,  84, 255],
         ...,
         [ 38,  52,  72, 253],
         [ 32,  42,  71, 255],
    

In [8]:
ssim_scores = []

for og_fragment in tqdm(og_fragments):
    for reference in references: 
        max_w = max(og_fragment.shape[0], reference.shape[0])
        max_h = max(og_fragment.shape[1], reference.shape[1])
        
        before = cv.resize(og_fragment, (max_w, max_h))
        after = cv.resize(reference, (max_w, max_h))
        
        # Convert images to grayscale
        before_gray = cv.cvtColor(before, cv.COLOR_BGR2GRAY)
        after_gray = cv.cvtColor(after, cv.COLOR_BGR2GRAY)
        (score, diff) = structural_similarity(before_gray, after_gray, full=True)
        ssim_scores.append(score)
ssim_scores

100%|██████████| 328/328 [14:10<00:00,  2.59s/it]


[0.2915017616183379,
 0.32547046873408886,
 0.23797627864953716,
 0.11933167828913267,
 0.24419996974959474,
 0.59436742306599,
 0.5763077747769388,
 0.09179207235743958,
 0.28264074701493475,
 0.33073544030500523,
 0.22009364364895356,
 0.11374077252611563,
 0.24958196488864381,
 0.5682271869148958,
 0.5487630654972088,
 0.08534371315128443,
 0.25606560335291534,
 0.2959444060107882,
 0.22215171021608804,
 0.12651312749924834,
 0.2029737206703889,
 0.5332278383397726,
 0.5174923180180424,
 0.08891224102729227,
 0.24934920956350054,
 0.2855068869023088,
 0.24755018400152085,
 0.1325119021921729,
 0.17177251577633318,
 0.5147125580528293,
 0.48990441683785463,
 0.08952746582807034,
 0.235488059187629,
 0.26618667757725967,
 0.25900437877458343,
 0.13940275162205332,
 0.1528068921783286,
 0.471760660955438,
 0.45574236574887067,
 0.09444447860982261,
 0.27698906608623497,
 0.31350252530697525,
 0.2341337676580006,
 0.12379347482835777,
 0.23077078497090128,
 0.5764297092968605,
 0.555963

In [22]:
len(ssim_scores)

2624

In [9]:
flatten_color_histograms = compute_color_histograms(fragments)
unflatten_color_histograms = compute_color_histograms(fragments, flatten=False)

Computing color histograms: 100%|██████████| 328/328 [00:00<00:00, 65479.85it/s]


In [45]:
X = []
for idx, histogram in enumerate(flatten_color_histograms): 
    X.append(np.concatenate((histogram, np.array(ssim_scores[idx * len(references):(idx + 1) * len(references)]))))
X

[array([0.04166667, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.08333334,
        0.04166667, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [32]:
X = np.asarray(X, dtype="object")
X.shape

(328, 2)

In [47]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=len(references), random_state=42)
fit_kmeans = kmeans.fit(X)
create_cluster_dirs(data_dir="./data", output_dir="clusters/kmeans/colors_ssim", labels=fit_kmeans.labels_)

Creating cluster dirs: 100%|██████████| 328/328 [00:00<00:00, 1006.34it/s]


In [78]:
scores = {}
for reference_id in references_ids:
    scores[reference_id] = compute_metrics2(reference_id, "clusters/kmeans/colors_ssim")

threshold = 0.80
opt_clusters = {}
for reference_id, d in scores.items():
    max_items = d["max_items"]
    for max_item in max_items:
        if max_item[1] >= threshold:
            if reference_id in opt_clusters:
                opt_clusters[reference_id].append(max_item)
            else:
                opt_clusters[reference_id] = [max_item]
opt_clusters

{'34': [('cluster_5', 0.8405797101449275)],
 '35': [('cluster_4', 0.8974358974358975)],
 '37': [('cluster_2', 0.9135802469135802)],
 '39': [('cluster_1', 0.8333333333333333)]}